# Recommender System Evaluations - Part 2
> Understanding evaluation metrics and pricing factors

- toc: true
- badges: true
- comments: true
- categories: [Evaluation]
- image:

In [1]:
import  numpy  as  np 
import  pandas  as  pd 
import  math

## HR@K

In [2]:
def hit_rate_at_k(recommended_list, bought_list, k=5): 
  bought_list = np.array(bought_list) 
  recommended_list = np.array(recommended_list)[:k]
  flags = np.isin(bought_list, recommended_list) 
  return (flags.sum() > 0) * 1

In [4]:
recommended_list = [156, 1134, 27, 1543, 3345, 143, 32, 533, 11, 43]  #items ids
bought_list = [521, 32, 143, 991]

In [6]:
hit_rate_at_k(recommended_list, bought_list, 5)

0

In [9]:
hit_rate_at_k(recommended_list, bought_list, 10)

1

## Precision@K

- Precision = (# of recommended items that are relevant) / (# of recommended items)
- Precision @ k = (# of recommended items @k that are relevant) / (# of recommended items @k)
- Money Precision @ k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)

In [11]:
def precision_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)[:k]
  
  flags = np.isin(bought_list, recommended_list)
  return flags.sum() / len(recommended_list)

In [10]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
  recommend_list = np.array(recommended_list)[:k] 
  prices_recommended = np.array(prices_recommended)[:k]
  flags = np.isin(recommend_list, bought_list)
  precision = np.dot(flags, prices_recommended) / prices_recommended.sum()
  return precision

In [12]:
recommended_list = [156, 1134, 27, 1543, 3345, 143, 32, 533, 11, 43]  #items ids
bought_list = [521, 32, 143, 991]
prices_recommendede_list = [400, 60, 40, 90, 60, 340, 70, 190,110, 240]

In [14]:
precision_at_k(recommended_list, bought_list, 5)

0.0

In [15]:
precision_at_k(recommended_list, bought_list, 10)

0.2

In [16]:
money_precision_at_k(recommended_list, bought_list, prices_recommendede_list, 5)

0.0

In [17]:
money_precision_at_k(recommended_list, bought_list, prices_recommendede_list, 10)

0.25625

## Recall@K

- Recall = (# of recommended items that are relevant) / (# of relevant items)
- Recall @ k = (# of recommended items @k that are relevant) / (# of relevant items)
- Money Recall @ k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)

In [21]:
recommended_list=[143,156,1134,991,27,1543,3345,533,11,43] #itemsid
prices_recommended_list=[400,60,40,90,60,340,70,190,110,240]

bought_list=[521,32,143,991]
prices_bought=[150,30,400,90]

In [19]:
def recall_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)[:k]
  
  flags = np.isin(bought_list, recommended_list)
  return flags.sum() / len(bought_list)

In [20]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
  bought_list = np.array(bought_list)
  prices_bought = np.array(prices_bought)
  recommended_list = np.array(recommended_list)[:k]
  prices_recommended = np.array(prices_recommended)[:k]

  flags = np.isin(recommended_list, bought_list)
  return np.dot(flags, prices_recommended)/prices_bought.sum()

In [22]:
recall_at_k(recommended_list, bought_list, 5)

0.5

In [23]:
recall_at_k(recommended_list, bought_list, 10)

0.5

In [24]:
money_recall_at_k(recommended_list, bought_list, prices_recommendede_list, 5)

98.0

In [25]:
money_recall_at_k(recommended_list, bought_list, prices_recommendede_list, 10)

49.0

## MAP@K
- MAP @ k (Mean Average Precision @ k )
- Average AP @ k for all users

In [26]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [27]:
def map_k(recommended_list, bought_list, k=5):

    amount_user = len(bought_list)
    list_ap_k = [ap_k(recommended_list[i], bought_list[i], k) for i in np.arange(amount_user)]
    
    sum_ap_k = sum(list_ap_k)  
    return sum_ap_k/amount_user

In [28]:
#list of 3 users
recommended_list_3_users = [[143,156,1134,991,27,1543,3345,533,11,43],
                            [1134,533,14,4,15,1543,1,99,27,3345],
                            [991,3345,27,533,43,143,1543,156,1134,11]]

bought_list_3_users= [[521,32,143], #user1
                      [143,156,991,43,11], #user2
                      [1,2]] #user3

In [29]:
map_k(recommended_list_3_users, bought_list_3_users, 5)

0.3333333333333333

## MRR@K

In [31]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    recommended_list = np.array(recommended_list)
    bought_list = np.array(bought_list)
    
    amount_user = len(bought_list)
    rr = []
    for i in np.arange(amount_user):    
        relevant_indexes = np.nonzero(np.isin(recommended_list[i][:k], bought_list[i]))[0]
        if len(relevant_indexes) != 0:
            rr.append(1/(relevant_indexes[0]+1))
    
    if len(rr) == 0:
        return 0
    
    return sum(rr)/amount_user

In [33]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, 5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


0.3333333333333333

## NDCG@K

In [34]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    rec = recommended_list
    b = bought_list
    
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    flags = np.isin(recommended_list, bought_list)
    rank_list = []
    for i in np.arange(len(recommended_list)):
        if i < 2:
            rank_list.append(i+1)
        else:
            rank_list.append(math.log2(i+1))
    if len(recommended_list) == 0:
        return 0
    dcg = sum(np.divide(flags, rank_list)) / len(recommended_list)

    i_dcg = sum(np.divide(1, rank_list)) / len(recommended_list)
#     print(i_dcg)
    return dcg/i_dcg

In [35]:
recommended_list = [143,156,1134,991,27,1543,3345,533,11,43] #iditems
prices_recommended_list = [400,60,40,90,60,340,70,190,110,240]

bought_list = [521,32,143,991]
prices_bought = [150,30,400,90]

In [37]:
ndcg_at_k(recommended_list, bought_list, 5)

0.489938890671454